<a href="https://colab.research.google.com/github/shreekrishh/Data-Analytics/blob/main/language%20modelling%20using%20BertForMLM/language_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Oct 29 13:49:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#importing drive 
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 24.8 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 45.3 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#importing all necessary libraries
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from torch.nn import functional as F
import torch
from transformers import pipeline
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader
import nltk
import re
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
device = 'cuda'

In [ ]:
# importing the text file with the encoding
with open('/gdrive/MyDrive/language modelling/COMBINED (1).txt', 'r', encoding='utf-8') as f:
  string = f.read()

In [ ]:
#create a function to drop the last word (here to be gonna predicted) of string 
def drop_last_word(string):
  return string.rsplit(' ',1)[0]

In [ ]:
# Preprocessing text

def preprocess_text(string):

        text = string.lower()                         #convert the text into lower alphabets
        white = re.compile(r"\n")                     #removing spaces from the string
        text = white.sub(repl="",string = text)
        newline = re.compile(r"\n")                   #removing extra lines 
        text = newline.sub(repl="",string = text)
        double_newline = re.compile(r"\n\n")
        text = double_newline.sub(repl="",string = text)     
        text = text.strip(" ")                        #removing spaces between the quotaions and words
        html = re.compile(r'<.*?>')
        text = html.sub(r"",string = text)             #removing html links
        text = re.sub(r'http\S+', '', text)

        sub = sent_tokenize(text)

        for i in tqdm(range(len(sub))):
            pattern = r'[0-9]'                          #removing numbers
            sub[i] = re.sub(pattern, '', sub[i])
            sub[i] = re.sub(r'[^\w\s]', '', sub[i])     #removing punctuation
            sub[i] = sub[i] + '.'                       #adding period

        ignore = []
        for i in range(len(sub)):
            if len(sub[i].split())<3:
                sub[i] = "".join(sub[i] + 'IGNORE IGNORE IGNORE')

            if len(sub[i].split())>450:
                ignore.append(i)
                sub[i] = " ".join(sub[i].split()[:450] + ['.'])
        
        return sub, ignore

sub, ignore = preprocess_text(string)

100%|██████████| 52749/52749 [00:00<00:00, 106934.09it/s]


In [ ]:

count = 0
for i in ignore:
  del sub[i-count]
  count += 1
len(sub)

52735

In [ ]:
#replacing a mask in place of each target word (that is gonna be predicted)
masked = []
for i in range(len(sub)):
  masked.append(drop_last_word(sub[i]))
  masked[i] += ' [MASK].'
masked[0]

' john f helliwell richard layard jeffrey d sachs janemmanuel de neve lara b aknin and shun wang \x0cthe world happiness report was written by a group of independent experts acting in their personal [MASK].'

In [ ]:
df = pd.DataFrame({'sentence': sub})
df['masked sentence'] = masked

In [ ]:
#fucntion for assigning last word of a string as a target word
def target_word(sent):
  return sent.rsplit(' ',1)[1][:-1]

In [ ]:
#creating list for target words of all sentences
target = []
for i in range(len(sub)):
  target.append(target_word(sub[i]))
df['Target Words'] = target

In [ ]:
df['predictions'] = ''
df.to_csv("/gdrive/MyDrive/language modelling/sentence_data.csv")

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/language modelling/sentence_data.csv')

In [ ]:
display(df)

,Unnamed: 0,sentence,masked sentence,Target Words,predictions
0,0,john f helliwell richard layard jeffrey d sac...,john f helliwell richard layard jeffrey d sac...,capacities,NaN
1,1,any views expressed in this report do not nece...,any views expressed in this report do not nece...,nations,NaN
2,2,this publication may be reproduced using the f...,this publication may be reproduced using the f...,eds,NaN
3,3,world happiness report new york sustainable ...,world happiness report new york sustainable ...,network,NaN
4,4,full text and supporting documentation can be ...,full text and supporting documentation can be ...,solving,NaN
...,...,...,...,...,...
52730,52730,this caused me a lot of bewilderment is it po...,this caused me a lot of bewilderment is it po...,now,NaN
52731,52731,four days ago i got an email from amazon notif...,four days ago i got an email from amazon notif...,wasnt,NaN
52732,52732,consequently i called amazon to let them know ...,consequently i called amazon to let them know ...,refunded,NaN
52733,52733,i asked for an email documenting the phone con...,i asked for an email documenting the phone con...,day,NaN


In [ ]:
#inherting tokenizer from pretrained bert-base-uncased which contains most of the methods
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',    return_dict = True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
masked_sentences = df['masked sentence']

In [ ]:
#laoding model in cuda device
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
pred_words_list = []

In [ ]:
def get_predictions():

        for sentence in tqdm(masked_sentences):

                #print('Length:', len(sentence.split()))
                input = tokenizer.encode_plus(sentence, return_tensors = "pt", max_length = 512)            #tokenizing masked sentence
                input.to(device)
                mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)                  
                output = model(**input)                                                                     #output of the model
                logits = output.logits                                                                      #for Prediction of scores of the language modeling for each token
                softmax = F.softmax(logits, dim = -1)                                                     #last activation function for the normalization of the probablility function
                #print('Mask index', mask_index)
                if mask_index[0].size() == 0:
                    continue
                else:
                    mask_word = softmax[0, mask_index[0], :]
                    #print(mask_word.size())
                    if mask_word.size()[0] == 0:
                        continue
                    else:
                        top = torch.topk(mask_word, 1, dim = 1)[1][0]                                      #predicting the token having highest probability 
                        word = tokenizer.decode([top[0]])                                                  #predicting word
                pred_words_list.append(word)

get_predictions()

 43%|████▎     | 22832/52735 [08:21<11:00, 45.28it/s]

In [ ]:
#calculating accuracy
target_words = df['Target Words']
accuracy = 0
for i in range(len(target_words)):
        if pred_words_list[i] == target_words[i]:
            accuracy += 1
accuracy = accuracy / len(target_words)
print("Accuracy of the language model is ", 1 - accuracy)

Accuracy of the language model is  0.996624632596947


In [ ]:
df['predictions'] = pred_words_list
df.to_csv("/gdrive/MyDrive/language modelling/sentence_data.csv")